In [3]:
import pandas as pd

# Load the file without assumptions
df = pd.read_csv("../data/processed/full_SPY_data.csv")

# Show column names
print("Columns:", df.columns.tolist())

# Show a few rows
df.head()

Columns: ['volume', 'vw', 'open', 'close', 'high', 'low', 'timestamp', 'trades']


,volume,vw,open,close,high,low,timestamp,trades
0,4927.0,294.0346,293.83,293.96,294.34,293.83,2020-05-11 08:00:00,48
1,7420.0,293.9817,294.01,293.92,294.04,293.92,2020-05-11 08:05:00,46
2,3675.0,293.8851,293.95,293.60,293.98,293.59,2020-05-11 08:10:00,35
3,6945.0,293.5993,293.60,293.67,293.69,293.57,2020-05-11 08:15:00,35
4,4252.0,293.7026,293.68,293.55,293.76,293.55,2020-05-11 08:20:00,45


In [4]:
import pandas as pd

# Load the main data file
df = pd.read_csv("../data/processed/full_SPY_data.csv")

# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Extract date and time as new columns
df["date"] = df["timestamp"].dt.date
df["time"] = df["timestamp"].dt.strftime("%H:%M")

# Preview
print("Columns now:", df.columns.tolist())
df.head()

Columns now: ['volume', 'vw', 'open', 'close', 'high', 'low', 'timestamp', 'trades', 'date', 'time']


,volume,vw,open,close,high,low,timestamp,trades,date,time
0,4927.0,294.0346,293.83,293.96,294.34,293.83,2020-05-11 08:00:00,48,2020-05-11,08:00
1,7420.0,293.9817,294.01,293.92,294.04,293.92,2020-05-11 08:05:00,46,2020-05-11,08:05
2,3675.0,293.8851,293.95,293.60,293.98,293.59,2020-05-11 08:10:00,35,2020-05-11,08:10
3,6945.0,293.5993,293.60,293.67,293.69,293.57,2020-05-11 08:15:00,35,2020-05-11,08:15
4,4252.0,293.7026,293.68,293.55,293.76,293.55,2020-05-11 08:20:00,45,2020-05-11,08:20


In [5]:
# Filter just the 9:30 and 9:35 rows
target_times = ["09:30", "09:35"]
filtered_df = df[df["time"].isin(target_times)].copy()

# Verify
print("Filtered rows:", filtered_df.shape[0])
filtered_df.head()

Filtered rows: 4856


,volume,vw,open,close,high,low,timestamp,trades,date,time
18,5770.0,291.8525,291.76,291.85,291.97,291.75,2020-05-11 09:30:00,45,2020-05-11,09:30
19,4901.0,291.9880,291.86,291.94,292.05,291.86,2020-05-11 09:35:00,37,2020-05-11,09:35
400,22012.0,285.9674,285.96,286.14,286.14,285.82,2020-05-13 09:30:00,105,2020-05-13,09:30
401,12385.0,286.3869,286.23,286.39,286.47,286.23,2020-05-13 09:35:00,58,2020-05-13,09:35
592,6000.0,281.7238,281.62,281.69,281.81,281.62,2020-05-14 09:30:00,21,2020-05-14,09:30


In [7]:
# Step 1: Pivot 9:30 and 9:35 rows to columns

# Drop duplicate (date, time) rows to avoid pivot error
filtered_df = filtered_df.drop_duplicates(subset=["date", "time"])

# Then pivot as before
pivot = filtered_df.pivot(index="date", columns="time", values="close")


# Step 2: Compute the target
# 1 = buy at 9:30, sell at 9:35 (profitable); 0 = otherwise
pivot["target"] = (pivot["09:35"] > pivot["09:30"]).astype(int)

# Step 3: Flatten back to a DataFrame
labels_df = pivot[["target"]].reset_index()

# Preview
print(labels_df.shape)
labels_df.head()

(1238, 2)


time,date,target
0,2020-05-11,1
1,2020-05-13,1
2,2020-05-14,1
3,2020-05-15,0
4,2020-05-18,1


In [33]:
# Block 1: Price-Based Features
block1 = df.copy()

block1["range_pct"] = (block1["high"] - block1["low"]) / block1["vw"]
block1["mad_diff"] = abs(block1["close"] - block1["vw"])
block1["candle_body"] = abs(block1["close"] - block1["open"])

# Keep only relevant columns
block1 = block1[["date", "range_pct", "mad_diff", "candle_body"]]

# Preview
print("Block 1 (Price-Based Features):")
block1.head()

Block 1 (Price-Based Features):


,date,range_pct,mad_diff,candle_body
0,2020-05-11,0.001734,0.0746,0.13
1,2020-05-11,0.000408,0.0617,0.09
2,2020-05-11,0.001327,0.2851,0.35
3,2020-05-11,0.000409,0.0707,0.07
4,2020-05-11,0.000715,0.1526,0.13


In [34]:
# Block 2: Rolling Averages & Ratios
block2 = df.copy()

block2["sma_3"] = block2["vw"].rolling(window=3).mean()
block2["sma_ratio"] = block2["vw"] / block2["sma_3"]
block2["sma_distance"] = abs(block2["vw"] - block2["sma_3"])

# Keep only relevant columns
block2 = block2[["date", "sma_3", "sma_ratio", "sma_distance"]]

# Preview
print("Block 2 (SMA Features):")
block2.head()

Block 2 (SMA Features):


,date,sma_3,sma_ratio,sma_distance
0,2020-05-11,NaN,NaN,NaN
1,2020-05-11,NaN,NaN,NaN
2,2020-05-11,293.967133,0.999721,0.082033
3,2020-05-11,293.822033,0.999242,0.222733
4,2020-05-11,293.729000,0.999910,0.026400


In [35]:
# Block 3: Previous Day Summary Features
block3 = df.copy()

prev_df = block3.groupby("date").agg({
    "high": "max",
    "low": "min",
    "close": "last"
}).rename(columns={
    "high": "prev_day_high",
    "low": "prev_day_low",
    "close": "prev_day_close"
})

# Shift forward so today sees yesterday’s values
prev_df = prev_df.shift(1)

# Merge into block3
block3 = block3.merge(prev_df, on="date", how="left")

# Derived features
block3["prev_day_range"] = block3["prev_day_high"] - block3["prev_day_low"]
block3["prev_day_change"] = (block3["open"] - block3["prev_day_close"]) / block3["prev_day_close"]

# Keep only relevant columns
block3 = block3[["date", "prev_day_high", "prev_day_low", "prev_day_close", "prev_day_range", "prev_day_change"]]

# Preview
print("Block 3 (Previous Day Summary Features):")
block3.head()

Block 3 (Previous Day Summary Features):


,date,prev_day_high,prev_day_low,prev_day_close,prev_day_range,prev_day_change
0,2020-05-11,NaN,NaN,NaN,NaN,NaN
1,2020-05-11,NaN,NaN,NaN,NaN,NaN
2,2020-05-11,NaN,NaN,NaN,NaN,NaN
3,2020-05-11,NaN,NaN,NaN,NaN,NaN
4,2020-05-11,NaN,NaN,NaN,NaN,NaN


In [36]:
# Block 4: Volatility & Bollinger Band Features
block4_source = df[df["time"] == "09:30"].copy()
block4_source = block4_source[["date", "open"]].rename(columns={"open": "open_930"})

# Compute features
block4_source["prior_volatility"] = block4_source["open_930"].rolling(window=5).std()
block4_source["sma_20"] = block4_source["open_930"].rolling(window=20).mean()
block4_source["std_20"] = block4_source["open_930"].rolling(window=20).std()

# Bollinger Bands
block4_source["bollinger_upper"] = block4_source["sma_20"] + 2 * block4_source["std_20"]
block4_source["bollinger_lower"] = block4_source["sma_20"] - 2 * block4_source["std_20"]
block4_source["bollinger_width"] = block4_source["bollinger_upper"] - block4_source["bollinger_lower"]
block4_source["bollinger_position"] = block4_source["open_930"] - block4_source["bollinger_lower"]

# Keep only needed columns
block4 = block4_source[["date", "prior_volatility", "bollinger_width", "bollinger_position"]]

# Preview
print("Block 4 (Volatility & Bollinger Band Features):")
block4.head()

Block 4 (Volatility & Bollinger Band Features):


,date,prior_volatility,bollinger_width,bollinger_position
18,2020-05-11,NaN,NaN,NaN
400,2020-05-13,NaN,NaN,NaN
592,2020-05-14,NaN,NaN,NaN
784,2020-05-15,NaN,NaN,NaN
976,2020-05-18,3.805559,NaN,NaN


In [37]:
# Block 4b: Overnight Gap Feature

# Step 1: Get 9:35 close and compute previous close
close_935 = filtered_df[filtered_df["time"] == "09:35"].copy()
close_935["prev_close"] = close_935["close"].shift(1)

# Step 2: Get 9:30 open prices
open_930 = filtered_df[filtered_df["time"] == "09:30"].copy()

# Step 3: Merge close data into open data
open_930 = open_930.merge(close_935[["date", "prev_close"]], on="date", how="left")

# Step 4: Calculate overnight gap
open_930["overnight_gap"] = (open_930["open"] - open_930["prev_close"]) / open_930["prev_close"]

# Step 5: Create block
block4b = open_930[["date", "overnight_gap"]]

# Preview
print("Block 4b (Overnight Gap Feature):")
block4b.head()

Block 4b (Overnight Gap Feature):


,date,overnight_gap
0,2020-05-11,NaN
1,2020-05-11,-0.480503
2,2020-05-13,-0.020484
3,2020-05-13,-0.020484
4,2020-05-14,-0.016656


In [43]:
# Step 1: Calculate RSI and daily return again (to reattach them)
# Reuse filtered_df
close_935 = filtered_df[filtered_df["time"] == "09:35"].copy()
close_935["daily_return"] = close_935["close"].pct_change()

delta = close_935["close"].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
close_935["rsi_14"] = 100 - (100 / (1 + rs))

# Step 2: Shift 9:35 close to get prev_close
close_935["prev_close"] = close_935["close"].shift(1)

# Step 3: Prepare open_930
open_930 = filtered_df[filtered_df["time"] == "09:30"].copy()

# Step 4: Merge daily_return and rsi into open_930
open_930 = open_930.merge(close_935[["date", "rsi_14", "daily_return", "prev_close"]], on="date", how="left")

# Step 5: Calculate overnight_gap
open_930["overnight_gap"] = (open_930["open"] - open_930["prev_close"]) / open_930["prev_close"]

# Step 6: Finalize block5
block5 = open_930[["date", "rsi_14", "daily_return", "overnight_gap"]].dropna().reset_index(drop=True)

# Preview
print("Final Block 5:")
block5.head()

Final Block 5:


,date,rsi_14,daily_return,overnight_gap
0,2020-05-11,17.370603,-0.480182,-0.480503
1,2020-05-13,13.636755,-0.019011,-0.020484
2,2020-05-14,13.432413,-0.016237,-0.016656
3,2020-05-15,14.532882,0.013168,0.014268
4,2020-05-18,15.548407,0.012437,0.012331


In [46]:
#Block 6.  Work from the same 9:30 open prices


# Create trend-based features
open_930["open_prev_1"] = open_930["open"].shift(1)
open_930["open_prev_2"] = open_930["open"].shift(2)

open_930["1d_trend"] = (open_930["open"] - open_930["open_prev_1"]) / open_930["open_prev_1"]
open_930["2d_trend"] = (open_930["open"] - open_930["open_prev_2"]) / open_930["open_prev_2"]

# Create gap vs trend ratio
open_930["gap_vs_trend"] = open_930["overnight_gap"] / open_930["1d_trend"]

# Final trend features block
block6 = open_930[["date", "1d_trend", "2d_trend", "gap_vs_trend"]].copy()

# Preview
print("Block 6 (Trend and Gap Features):")
block6.head()

Block 6 (Trend and Gap Features):


,date,1d_trend,2d_trend,gap_vs_trend
0,2020-05-11,NaN,NaN,NaN
1,2020-05-11,0.000000,NaN,-inf
2,2020-05-13,-0.019879,-0.019879,1.030399
3,2020-05-13,0.000000,-0.019879,-inf
4,2020-05-14,-0.015177,-0.015177,1.097428


In [47]:
# Block 7: Calendar Features
calendar = open_930[["date"]].copy()
calendar["date"] = pd.to_datetime(calendar["date"])

# Day of week: Monday=0, Sunday=6
calendar["day_of_week"] = calendar["date"].dt.weekday
calendar["is_monday"] = (calendar["day_of_week"] == 0).astype(int)
calendar["is_friday"] = (calendar["day_of_week"] == 4).astype(int)
calendar["month"] = calendar["date"].dt.month

# Final calendar block
block7 = calendar[["date", "day_of_week", "is_monday", "is_friday", "month"]].copy()

# Preview
print("Block 7 (Calendar Features):")
block7.head()

Block 7 (Calendar Features):


,date,day_of_week,is_monday,is_friday,month
0,2020-05-11,0,1,0,5
1,2020-05-11,0,1,0,5
2,2020-05-13,2,0,0,5
3,2020-05-13,2,0,0,5
4,2020-05-14,3,0,0,5


In [48]:
# Block 8: Market Trend Features
trend = open_930[["date", "open"]].copy()

# Rolling trend over the past 5, 10, and 20 days
trend["trend_5"] = trend["open"].pct_change(periods=5)
trend["trend_10"] = trend["open"].pct_change(periods=10)
trend["trend_20"] = trend["open"].pct_change(periods=20)

# Final trend block
block8 = trend[["date", "trend_5", "trend_10", "trend_20"]].copy()

# Preview
print("Block 8 (Market Trend Features):")
block8.head()

Block 8 (Market Trend Features):


,date,trend_5,trend_10,trend_20
0,2020-05-11,NaN,NaN,NaN
1,2020-05-11,NaN,NaN,NaN
2,2020-05-13,NaN,NaN,NaN
3,2020-05-13,NaN,NaN,NaN
4,2020-05-14,NaN,NaN,NaN


In [49]:
# Block 9: Merge all blocks into one final DataFrame
from functools import reduce

# List of all blocks to merge on 'date'
blocks = [block1, block2, block3, block4, block5, block6, block7, block8]

# Merge all feature blocks
features_df = reduce(lambda left, right: pd.merge(left, right, on="date", how="inner"), blocks)

# Merge with labels
final_df = pd.merge(features_df, labels_df, on="date", how="inner")

# Drop any rows with missing values (optional but recommended)
final_df = final_df.dropna().reset_index(drop=True)

# Preview final shape and sample
print("Final dataset shape:", final_df.shape)
final_df.head()

MemoryError: Unable to allocate 608. GiB for an array with shape (81593359880,) and data type int64